In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/10-monkey-species/monkey_labels.txt
/kaggle/input/10-monkey-species/validation/validation/n7/n711.jpg
/kaggle/input/10-monkey-species/validation/validation/n7/n703.jpg
/kaggle/input/10-monkey-species/validation/validation/n7/n7015.jpg
/kaggle/input/10-monkey-species/validation/validation/n7/n7017.jpg
/kaggle/input/10-monkey-species/validation/validation/n7/n709.jpg
/kaggle/input/10-monkey-species/validation/validation/n7/n712.jpg
/kaggle/input/10-monkey-species/validation/validation/n7/n710.jpg
/kaggle/input/10-monkey-species/validation/validation/n7/n7012.jpg
/kaggle/input/10-monkey-species/validation/validation/n7/n713.jpg
/kaggle/input/10-monkey-species/validation/validation/n7/n7011.jpg
/kaggle/input/10-monkey-species/validation/validation/n7/n7016.jpg
/kaggle/input/10-monkey-species/validation/validation/n7/n7010.jpg
/kaggle/input/10-monkey-species/validation/validation/n7/n706.jpg
/kaggle/input/10-monkey-species/validation/validation/n7/n717.jpg
/kaggle/input/10-mon

In [2]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions
from keras.models import Sequential
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout,GlobalAveragePooling2D

In [3]:
from keras.applications import MobileNet
img_rows , img_cols = 224,224
MobileNet = MobileNet(weights = 'imagenet',
                     include_top = False,
                     input_shape =(img_rows,img_cols,3))

for layer in MobileNet.layers:
    layer.trainable = False

MobileNet.output

17227776/17225924 [==============================] - 0s 0us/step


<tf.Tensor 'conv_pw_13_relu/Relu6:0' shape=(None, 7, 7, 1024) dtype=float32>

In [4]:
from keras.layers import Dense, GlobalAveragePooling2D
def addTopMobileNet(bottom_model, num_classes):
    classifier = bottom_model.output
    classifier = GlobalAveragePooling2D()(classifier)
    classifier = Dense(units = 1024, activation = 'relu')(classifier)
    classifier = Dense(units = 256, activation = 'relu')(classifier)
    classifier = Dense(units = 128, activation = 'relu')(classifier)
    classifier = Dense(units = 10, activation = 'softmax')(classifier)
    return classifier

In [5]:
from keras.models import Model
num_classes = 10 
top = addTopMobileNet(MobileNet, num_classes)
model = Model(inputs = MobileNet.input, outputs = top)
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)     

In [6]:
#compile model using accuracy to measure model performance
model.compile(optimizer= 'adam', loss="categorical_crossentropy", metrics=['accuracy'])

In [7]:
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=15,
        zoom_range = 0.15,
        width_shift_range=0.15,
        height_shift_range=0.15
        )
test_datagen = ImageDataGenerator(rescale = 1./255)

In [8]:
batch_size = 16
training_set = train_datagen.flow_from_directory('/kaggle/input/10-monkey-species/training/training',
                                                 target_size=(224,224),
                                                 batch_size = batch_size,
                                                 class_mode='categorical')

Found 1098 images belonging to 10 classes.


In [9]:
test_set = test_datagen.flow_from_directory('/kaggle/input/10-monkey-species/validation/validation',
                                                 target_size=(224,224),
                                                  batch_size =8,
                                                 class_mode='categorical')

Found 272 images belonging to 10 classes.


In [10]:
from keras.callbacks import EarlyStopping 
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', patience = 10,restore_best_weights = True)
filepath="weights.hdf5"
cp_callback =ModelCheckpoint(filepath,
                            monitor='val_loss',
                            save_best_only=True,
                            verbose=1)

In [11]:
ls = [es,cp_callback]

In [12]:

history = model.fit_generator(training_set,
                              epochs = 10,
                              validation_data = test_set,
                              steps_per_epoch=1098/batch_size,
                              callbacks = ls)

Epoch 1/10
69/68 [==============================] - ETA: 0s - loss: 0.7549 - accuracy: 0.7404
Epoch 00001: val_loss improved from inf to 0.31266, saving model to weights.hdf5
69/68 [==============================] - 49s 716ms/step - loss: 0.7549 - accuracy: 0.7404 - val_loss: 0.3127 - val_accuracy: 0.8860
Epoch 2/10
69/68 [==============================] - ETA: 0s - loss: 0.1952 - accuracy: 0.9317
Epoch 00002: val_loss improved from 0.31266 to 0.14529, saving model to weights.hdf5
69/68 [==============================] - 44s 632ms/step - loss: 0.1952 - accuracy: 0.9317 - val_loss: 0.1453 - val_accuracy: 0.9485
Epoch 3/10
69/68 [==============================] - ETA: 0s - loss: 0.1369 - accuracy: 0.9526
Epoch 00003: val_loss improved from 0.14529 to 0.12630, saving model to weights.hdf5
69/68 [==============================] - 44s 635ms/step - loss: 0.1369 - accuracy: 0.9526 - val_loss: 0.1263 - val_accuracy: 0.9412
Epoch 4/10
69/68 [==============================] - ETA: 0s - loss: 0.0

In [17]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('/kaggle/input/10-monkey-species/validation/validation/n9/n9026.jpg', target_size=(224, 224))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)


In [18]:
import numpy as np
from keras import optimizers
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array

from keras.preprocessing import *

monkey_labels = {
    0: "Mantled Howler",
    1: "Patas Monkey",
    2: "Bald Uakari",
    3: "Japanese Macaque",
    4: "Pygmy Marmoset",
    5: "White-headed Capuchin",
    6: "Silvery Marmoset",
    7: "Common Squirrel Monkey",
    8: "Black-headed Night Monkey",
    9: "Nilgiri Langur",
}

# Prints a prediction
def print_prediction(best_guess, prob):
    print("Species: " + monkey_labels[best_guess])
    print("Probability: {0:.2%}\n".format(prob))


# Loads an image and makes a prediction using the model
def predict_image(filename):
    # Load image, downsize, scale and convert to array
    test_image = img_to_array(load_img(filename, target_size=(224,224))) / 255.0

    # Expand array by 1 to match model
    test_image = np.expand_dims(test_image, axis=0)

    # Calculate category probabilities using model
    predictions = model.predict(test_image)[0]

    # Get label of highest probability prediction
    best_guess = 0
    highest_prob = 0.0
    for i in range(len(predictions)):
        if predictions[i] > highest_prob:
            best_guess = i
            highest_prob = predictions[i]

    print_prediction(best_guess, highest_prob)


In [20]:
predict_image('/kaggle/input/10-monkey-species/validation/validation/n9/n9026.jpg')

Species: Nilgiri Langur
Probability: 100.00%

